In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("lab.ipynb")

# Lab 9 – Models and Pipelines 🔁

## DSC 80, Fall 2023

### Due Date: Monday, December 4th at 11:59PM

## Instructions
Welcome to the ninth lab assignment in DSC 80 this quarter!

Much like in DSC 10, this Jupyter Notebook contains the statements of the problems and provides code and Markdown cells to display your answers to the problems. Unlike DSC 10, the notebook is *only* for displaying a readable version of your final answers. The coding will be done in an accompanying `lab.py` file that is imported into the current notebook, and **you will only submit that `lab.py` file**, not this notebook!

Some additional guidelines:
- **Unlike in DSC 10, labs will have both public tests and hidden tests.** The bulk of your grade will come from your scores on hidden tests, which you will only see on Gradescope after the assignment deadline.
- **Do not change the function names in the `lab.py` file!** The functions in the `lab.py` file are how your assignment is graded, and they are graded by their name. If you changed something you weren't supposed to, you can find the original code in the [course GitHub repository](https://github.com/dsc-courses/dsc80-2023-wi).
- Notebooks are nice for testing and experimenting with different implementations before designing your function in your `lab.py` file. You can write code here, but make sure that all of your real work is in the `lab.py` file, since that's all you're submitting.
- **To ensure that all of your work to be submitted is in `lab.py`, we've provided an additional uneditable notebook, called `lab-validation.ipynb`, that contains only the tests and their setup. Make sure you are able to run it top-to-bottom without error before submitting!**
- You are encouraged to write your own additional helper functions to solve the lab, as long as they also end up in `lab.py`.

**Importing code from `lab.py`**:

* Below, we import the `.py` file that's contained in the same directory as this notebook.
* We use the `autoreload` notebook extension to make changes to our `lab.py` file immediately available in our notebook. Without this extension, we would need to restart the notebook kernel to see any changes to `lab.py` in the notebook.
    - `autoreload` is necessary because, upon import, `lab.py` is compiled to bytecode (in the directory `__pycache__`). Subsequent imports of `lab` merely import the existing compiled python.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from lab import *

In [4]:
import pandas as pd
import numpy as np
import os
import plotly.express as px

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from pipeline_testing_util import get_transformers

from plotly.subplots import make_subplots
import plotly.graph_objects as go

import warnings
warnings.simplefilter('ignore')

## Part 1: `sklearn` Pipelines 🧠

The file `data/toy.csv` contains an example dataset that consists of 4 columns:

- `'group'`: a categorical column with 3 categories
- `'c1'`: a numeric attribute
- `'c2'`: a numeric attribute
- `'y'`: the target variable (that you want to predict) 
```

In the following questions, you will build `Pipeline`s that combine feature engineering with a linear regression model.

In [5]:
fp = os.path.join('data', 'toy.csv')
data = pd.read_csv(fp)
data.head()

,group,c1,c2,y
0,B,4.058118,5.329582,12.649035
1,A,4.194945,3.839476,17.309083
2,A,2.246411,10.694666,15.695646
3,B,2.510912,6.414960,11.535752
4,B,3.194722,6.116839,14.954389


### Question 1

First, you will train a regression model using only a *log-scaled* `'c2'` variable. Create a `Pipeline` that:
1. log-scales `'c2'`, then
2. predicts `'y'` using a linear regression model (using your transformed `'c2'`).

That is, complete the implementation of the function `simple_pipeline`, which takes in a DataFrame like `data` and returns a **tuple** consisting of 
- An already-fit `Pipeline`, and
- An array containing the predictions your model makes on `data` (after being trained on `data`).

***Note:*** By "log", we're referring to the natural logarithm. In order to log-scale `'c2'`, you'll need to use a `FunctionTransformer`.

In [6]:
q1_fp = os.path.join('data', 'toy.csv')
q1_data = pd.read_csv(q1_fp)
q1_data

,group,c1,c2,y
0,B,4.058118,5.329582,12.649035
1,A,4.194945,3.839476,17.309083
2,A,2.246411,10.694666,15.695646
3,B,2.510912,6.414960,11.535752
4,B,3.194722,6.116839,14.954389
...,...,...,...,...
995,C,0.436382,2.718282,7.725205
996,A,3.121212,8.636007,17.852916
997,B,2.215070,7.197776,13.864179
998,C,2.002297,2.718282,7.826493


In [7]:
# q1_data['c2_log'] = np.log(q1_data['c2'])
log_transformer = FunctionTransformer(np.log, validate=True)
# c2_model_log = LinearRegression()
# c2_model_log.fit(q1_data[['c2_log']], q1_data['y'])
pl = Pipeline(steps=[
    ('log_scaler', log_transformer),
    ('regressor', LinearRegression())
])
pl.fit(q1_data[['c2']], q1_data['y'])
pl.predict(q1_data[['c2']])

array([12.62080258, 11.33240999, 15.35709202, 13.34904182, 13.16208199,
       12.58730638, 10.32010771,  9.97565958, 12.4341862 ,  9.97565958,
       11.00176451, 11.72576886, 13.29324648, 13.52069629, 12.69877755,
       16.56022602, 10.90515178, 11.59328864, 13.75185396, 10.13914047,
       16.45741691, 12.22981832, 10.03041175, 13.6458863 , 10.40933462,
        9.97565958, 12.6181805 , 13.2395772 , 17.80416143,  9.97565958,
       17.31864025, 11.70631644, 10.04917783, 13.54303459, 10.23465198,
        9.97565958,  9.97565958, 14.13802627, 15.54064641,  9.97565958,
        9.97565958, 15.66902784,  9.97565958, 14.88720712, 10.67109288,
       11.29811807, 13.80545472, 10.23529717, 10.40456903, 13.08402701,
        9.97565958,  9.97565958,  9.97565958,  9.97565958, 15.10329322,
       12.32722694, 13.95698297,  9.97565958,  9.97565958,  9.97565958,
        9.97565958, 11.05800145,  9.97565958,  9.97565958, 13.77397551,
       17.70287735, 10.31042233, 15.31536588, 16.33383322,  9.97

In [8]:
# q1_data['c2_log'] = np.log(q1_data['c2'])
# log_transformer = FunctionTransformer(np.log, validate=True)
# c2_model_log = LinearRegression()
# c2_model_log.fit(q1_data[['c2_log']], q1_data['y'])
# pl = Pipeline(steps=[
#     ('regressor', LinearRegression())
# ])
# pl.fit(q1_data[['c2_log']], q1_data['y'])
# pl.predict(q1_data[['c2_log']])

In [9]:
# don't change this cell, but do run it -- it is needed for the tests to work
q1_fp = os.path.join('data', 'toy.csv')
q1_data = pd.read_csv(q1_fp)
q1_pl, q1_preds = simple_pipeline(q1_data)

In [10]:
grader.check("q1")

q1 results: All test cases passed!

### Question 2

Now, you will engineer features from the other columns and use them to train a regression model.  Create a `Pipeline` that:
1. uses `'c1'` as is,
1. log-scales `'c2'`,
1. one-hot encodes `'group'`, and
1. predicts `'y'` using a linear regression model built on the three variables above. (Note that your model will have more than three "features", because one-hot encoding `'group'` will create multiple columns. Don't drop any of them.)

That is, complete the implementation of the function `multi_type_pipeline`, which takes in a DataFrame like `data` and returns a **tuple** consisting of
- An already-fit `Pipeline`, and
- An array containing the predictions your model makes on `data` (after being trained on `data`).

***Hint:*** Use `ColumnTransformer`, as we did in [Lecture 22](https://dsc80.com/resources/lectures/lec22/lec22.html#Planning-our-first-ColumnTransformer).

In [11]:
q2_fp = os.path.join('data', 'toy.csv')
q2_data = pd.read_csv(q2_fp)
q2_data

,group,c1,c2,y
0,B,4.058118,5.329582,12.649035
1,A,4.194945,3.839476,17.309083
2,A,2.246411,10.694666,15.695646
3,B,2.510912,6.414960,11.535752
4,B,3.194722,6.116839,14.954389
...,...,...,...,...
995,C,0.436382,2.718282,7.725205
996,A,3.121212,8.636007,17.852916
997,B,2.215070,7.197776,13.864179
998,C,2.002297,2.718282,7.826493


In [12]:
preproc = ColumnTransformer(
    transformers=[
        ('c2_log', log_transformer, ['c2']),
        ('ohe_group', OneHotEncoder(), ['group'])
    ]
)

In [13]:
pl = Pipeline([
    ('preprocessor', preproc), 
    ('lin-reg', LinearRegression())
])
pl.fit(q2_data.drop(columns='y'), q2_data['y'])


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('c2_log',
                                                  FunctionTransformer(func=<ufunc 'log'>,
                                                                      validate=True),
                                                  ['c2']),
                                                 ('ohe_group', OneHotEncoder(),
                                                  ['group'])])),
                ('lin-reg', LinearRegression())])

In [14]:
# don't change this cell, but do run it -- it is needed for the tests to work
q2_fp = os.path.join('data', 'toy.csv')
q2_data = pd.read_csv(q2_fp)
q2_pl, q2_preds = multi_type_pipeline(q2_data)

In [15]:
grader.check("q2")

q2 results: All test cases passed!

### Question 3

It seems like `'c1'` and `'c2'` have strong associations with the values of `'group'` (to see this, run the cell below). This suggests that group-wise scaling might make good features. 


Now, we want to standardize (i.e. z-score) both `'c1'` and `'c2'` **within each `'group'`** (`'A'`, `'B'`, and `'C'`). Unfortunately, there is no built-in transformer in `sklearn` that performs group-wise standardization, so **you will need to create your own transformer!**

Your job is to complete the implementation of the `StdScalerByGroup` transformer class, meaning that you need to implement the `fit` and `transform` methods, along with the constructor (`__init__`).
- The `StdScalerByGroup` transformer works on an input array/DataFrame `X` whose first column contains groups, and whose remaining columns are quantitative and need to be standardized (within each group).
- The `fit` method should determine the mean and standard deviation of each quantitative column within each group in the input data `X` and save them in the instance variable `grps_`. (For instance, one of the quantities you may calculate here is the standard deviation of `'c1'`, but only for the rows whose `'group'` is `'B'`.)
- The `transform` method should take in an input array/DataFrame `X`, standardize each quantitative column separately using the means and standard deviations stored in `grps_`, and return a DataFrame containing the transformed quantitative columns.


If you `fit` and `transform` a `StdScalerByGroup` transformer on the `toy` DataFrame (without the `'y'` column), you should get back a DataFrame with two columns, `'c1'` and `'c2'`, with groups stored in the index (if you end up creating a `MultiIndex`, that is fine).


***Notes:***
1. You may decide on whatever structure you'd like for the `grps_` variable. This question will be graded on the correctness of the output of your transformer. (Check the correctness of your work by checking the output by-hand!)    
2. At no point should you loop over the **rows** of `data` (in fact, our solution doesn't use any loops), but you are allowed to use loop in this question.
3. The `'group'` column in the public tests is named `'g'` instead of `'group'`. Remember, the first column will **always** contain the groups, even if the first column's name is something other than `'group'`.
4. Do not worry about cases where the standard deviation is equal to 0, the function `std` might work in this case.

In [16]:
# The scatter plot referenced at the start of Question 3
# This is not needed to answer the question, but motivates why we are standardizing
px.scatter(data, x='c1', y='y', color='group')

In [17]:
q2_data.groupby('group')['c1', 'c2'].agg(['mean', 'std']).to_dict()
q2_data

,group,c1,c2,y
0,B,4.058118,5.329582,12.649035
1,A,4.194945,3.839476,17.309083
2,A,2.246411,10.694666,15.695646
3,B,2.510912,6.414960,11.535752
4,B,3.194722,6.116839,14.954389
...,...,...,...,...
995,C,0.436382,2.718282,7.725205
996,A,3.121212,8.636007,17.852916
997,B,2.215070,7.197776,13.864179
998,C,2.002297,2.718282,7.826493


In [18]:
q2_data.groupby(q2_data.columns[0])['c1', 'c2'].agg(['mean', 'std'])

c1                  c2          
           mean       std      mean       std
group                                        
A      2.896205  1.080452  8.867695  4.925313
B      1.968927  1.001601  4.667865  1.683758
C      1.105136  1.094182  3.003989  1.341565

In [19]:
q2_data['group']

0      B
1      A
2      A
3      B
4      B
      ..
995    C
996    A
997    B
998    C
999    C
Name: group, Length: 1000, dtype: object

In [20]:
# don't change this cell, but do run it -- it is needed for the tests to work
# test fit 
q3_test_fit_cols = {'g': ['A', 'A', 'B', 'B'], 'c1': [1, 2, 2, 2], 'c2': [3, 1, 2, 0]}
q3_test_fit_X = pd.DataFrame(q3_test_fit_cols)
q3_test_fit_std = StdScalerByGroup().fit(q3_test_fit_X)

# test transform
q3_test_transform_cols = {'g': ['A', 'A', 'B', 'B'], 'c1': [1, 2, 3, 4], 'c2': [1, 2, 3, 4]}
q3_test_transform_X = pd.DataFrame(q3_test_transform_cols)
q3_test_transform_std = StdScalerByGroup().fit(q3_test_transform_X)
q3_test_transform_out = q3_test_transform_std.transform(q3_test_transform_X)

In [21]:
# don't change this cell, but do run it -- it is needed for the tests to work
q3_fit_data = pd.read_csv('data/toy.csv')

N = 2*10**6
a = np.random.choice(['A', 'B'], size=(N,1)).astype('object')
b = np.random.multivariate_normal([1, 2], [[1, 0],[0, 100]], size=N)
arr = np.hstack([a, b])
q3_transform_data = pd.DataFrame(arr)
q3_transform_data[1] = q3_transform_data[1].astype(float)
q3_transform_data[2] = q3_transform_data[2].astype(float)

In [22]:
q3_test_transform_out

,c1,c2
0,-0.707107,-0.707107
1,0.707107,0.707107
2,-0.707107,-0.707107
3,0.707107,0.707107


In [23]:
grader.check("q3")

q3 results: All test cases passed!

### Question 4

`Pipeline`s are supposed to help you easily try different model configurations. Complete the implementation of the function `eval_toy_model`, which returns a hard-coded **list of tuples** consisting of the (RMSE, $R^2$) of three different modeling `Pipeline`s, fit and evaluated on the entire input dataset `data`. The three different `Pipeline`s are:
1. The `Pipeline` in Question 1.
1. The `Pipeline` in Question 2.
1. A `Pipeline` consisting of a linear regression model fit on features generated by applying `StdScalerByGroup` to `'c1'`, log-scaling `'c2'`, and applying `OneHotEncoder` to `'group'`.

In [24]:
toy_fp = os.path.join('data', 'toy.csv')
toy_data = pd.read_csv(toy_fp)
toy_pl1, toy_preds1 = simple_pipeline(toy_data)
toy_pl2, toy_preds2 = multi_type_pipeline(toy_data)

In [25]:
from sklearn.metrics import mean_squared_error, r2_score
rmse = np.sqrt(mean_squared_error(q2_data['y'], toy_preds2))
r_squared = r2_score(q2_data['y'], toy_preds2)
(rmse, r_squared)

(2.3148336164355268, 0.573324931567333)

In [26]:
from sklearn.metrics import mean_squared_error, r2_score

log_transformer = FunctionTransformer(np.log, validate=True)
preproc = ColumnTransformer(
    transformers=[
        ('c1_stdscaler', StdScalerByGroup(), ['group','c1']),
        ('c2_log', log_transformer, ['c2']),
        ('ohe_group', OneHotEncoder(), ['group'])
    ],
    remainder='passthrough'
)
pl3 = Pipeline([
    ('preprocessor', preproc), 
    ('lin-reg', LinearRegression())
])
results = []
pl3.fit(q2_data.drop('y', axis=1), q2_data['y'])
pl3_y_pred = pl3.predict(q2_data.drop('y', axis=1))
# Calculate RMSE and R²
rmse = np.sqrt(mean_squared_error(q2_data['y'], pl3_y_pred))
r_squared = r2_score(q2_data['y'], pl3_y_pred)

# Append the results
results.append((rmse, r_squared))
results

[(2.315733947782385, 0.5729929650348397)]

In [27]:
grader.check("q4")

q4 results: All test cases passed!

## Part 2: Overfitting 😟

### Question 5

In this question, you will train two different classes of prediction models – **decision tree and k-Nearest Neighbor regressors** – and explore different ways in which overfitting can appear. You'll use Galton's child heights dataset from the missingness lectures and Lab 5.

#### `tree_reg_perf` 🌲

A decision tree regressor is trained similar to a decision tree classifier: the splits of the tree are created by minimizing the variance of the (training data) response values in the leaves given by making the split in question. A decision tree regressor predicts the response value of a (new) observation based on the **average target value** of the training observations lying in the same leaf node. 

One **hyperparameter** of a decision tree regressor that affects model complexity is the **depth** of the tree. Larger depths correspond to more complicated decision trees. We will explore this parameter in this question.

Complete the implementation of the function `tree_reg_perf`, which takes in a DataFrame like `galton` and:
- Splits the data into training and test sets,
- Trains 20 decision trees – one for each depth between 1 and 20, and
- Computes both the training RMSE and testing RMSE of each tree.

Store and return your results in a DataFrame that has two columns, `'train_err'` and `'test_err'`, and an index that corresponds to tree depths (i.e. 1, 2, ..., 20).

<br>

#### `knn_reg_perf` 👉👈

A k-Nearest Neighbors (k-NN) regressor predicts the response value of a (new) observation by computing the average value of the k-closest observations in the training set. The most common distance metric is Euclidean distance, i.e. $L_2$ distance.

One **hyperparameter** of a k-NN regressor that affects model complexity is k, **the number of neighbors averaged over**. Larger values of k correspond to more complicated regressors. We will explore this hyperparameter in this question.

Complete the implementation of the function `knn_reg_perf`, which takes in a DataFrame like `galton` and:
- Splits the data into training and test sets,
- Trains 20 k-NN regressors – one for each value of k between 1 and 20, and
- Computes both the training RMSE and testing RMSE of each regressor.

Again, store and return your results in a DataFrame that has two columns, `'train_err'` and `'test_err'`, and an index that corresponds to values of k (i.e. 1, 2, ..., 20).

<br>

**Some guidelines for both subparts:**

- In all cases, we are using all other columns in `galton` to predict `'childHeight'`.
- You need to import the necessary classes from `sklearn` **inside** the functions you create. (Unlike before, we haven't imported them for you because we want you to figure out what to import!)
- If you're unsure how to create training and testing sets, refer to [Lecture 23](https://dsc80.com/resources/lectures/lec23/lec23.html). Use a test set size of 0.25. Within each function, you should only perform a single train-test split – that is, you should not be creating training and testing sets within a `for`-loop (though you will need to use a `for`-loop for something else).
    - For the purposes of this question, do not use any cross-validation.
- Don't write the formula for RMSE four times – define a helper function!

In [28]:
# Use `galton` to test your work.
galton = pd.read_csv('data/galton.csv')
galton.head()

,father,mother,children,childNum,gender,childHeight
0,78.5,67.0,4,1,1,73.2
1,78.5,67.0,4,2,0,69.2
2,78.5,67.0,4,3,0,69.0
3,78.5,67.0,4,4,0,69.0
4,75.5,66.5,4,1,1,73.5


In [29]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
X = galton.drop('childHeight', axis=1)
y = galton['childHeight']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
errors = {'train_err': [], 'test_err': []}
# Loop over tree depths from 1 to 20
for depth in range(1, 21):
    # Initialize the DecisionTreeRegressor with the current depth
    tree = DecisionTreeRegressor(max_depth=depth, random_state=42)

    # Train the model
    tree.fit(X_train, y_train)

    # Predict and calculate the RMSE for training set
    train_pred = tree.predict(X_train)
    train_err = np.sqrt(mean_squared_error(y_train, train_pred))
    errors['train_err'].append(train_err)

    # Predict and calculate the RMSE for test set
    test_pred = tree.predict(X_test)
    test_err = np.sqrt(mean_squared_error(y_test, test_pred))
    errors['test_err'].append(test_err)

# Create a DataFrame from the errors dictionary
results_df = pd.DataFrame(errors, index=range(1, 21))
results_df

,train_err,test_err
1,2.464331,2.617308
2,2.277170,2.524155
3,2.089859,2.357113
4,1.953536,2.257664
5,1.790380,2.158941
6,1.628651,2.261597
7,1.472174,2.339587
8,1.253968,2.434571
9,1.026741,2.442450
10,0.809883,2.628770


In [30]:
from sklearn.neighbors import KNeighborsRegressor

X = galton.drop('childHeight', axis=1)
y = galton['childHeight']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
errors = {'train_err': [], 'test_err': []}
# Loop over tree depths from 1 to 20
for k in range(1, 21):
    knn = KNeighborsRegressor(n_neighbors=k)
    # Train the model
    knn.fit(X_train, y_train)

    # Predict and calculate the RMSE for training set
    train_pred = knn.predict(X_train)
    train_err = np.sqrt(mean_squared_error(y_train, train_pred))
    errors['train_err'].append(train_err)

    # Predict and calculate the RMSE for test set
    test_pred = knn.predict(X_test)
    test_err = np.sqrt(mean_squared_error(y_test, test_pred))
    errors['test_err'].append(test_err)

# Create a DataFrame from the errors dictionary
results_df = pd.DataFrame(errors, index=range(1, 21))
results_df

,train_err,test_err
1,0.411742,2.826091
2,1.250642,2.637943
3,1.448085,2.439990
4,1.572848,2.327089
5,1.666598,2.277360
6,1.728301,2.298541
7,1.796734,2.319836
8,1.847978,2.303731
9,1.909552,2.244363
10,1.921027,2.271129


In [31]:
# don't change this cell, but do run it -- it is needed for the tests
galton_test = pd.read_csv('data/galton.csv')
out_tree_test = tree_reg_perf(galton_test)
out_knn_test = knn_reg_perf(galton_test)

In [32]:
grader.check("q5")

q5 results: All test cases passed!

After you've implemented both functions, run the cells below to plot training and testing error for both models.

In [33]:
np.random.seed(9) # For reproducibility

tree = tree_reg_perf(galton)
knn = knn_reg_perf(galton)
hyp = np.arange(1, 21)

fig = make_subplots(rows=1, cols=2, subplot_titles=('Error vs. Tree Depth<br>for Decision Tree Regressor',
                                                    'Error vs. k (# Neighbors)<br>for k-NN Regressor'))

fig.add_trace(
    go.Scatter(x=hyp, y=tree.iloc[:, 0], line=dict(color='blue'), name='Training Error'),
    row=1, col=1).add_trace(go.Scatter(x=hyp, y=tree.iloc[:, 1], line=dict(color='red'), name='Testing Error'), 
                            row=1, col=1)

fig.add_trace(
    go.Scatter(x=hyp, y=knn.iloc[:, 0], line=dict(color='blue'), name='Training Error', showlegend=False),
    row=1, col=2).add_trace(go.Scatter(x=hyp, y=knn.iloc[:, 1], line=dict(color='red'),  name='Testing Error',
                                       showlegend=False), row=1, col=2)

fig.update_layout(height=450, width=975)
fig.update_xaxes(title_text='Tree Depth', row=1, col=1, tickvals=np.arange(1,21,2))
fig.update_xaxes(title_text='k (# Neighbors)', row=1, col=2, tickvals=np.arange(1,21,2))

fig.show()

If your training and evaluation routines are correct, you should notice a few things:
- In both models, testing error initially decreases, and then (perhaps slowly) increases.
- With the decision tree, training error **decreases** as depth increases.
- With the k-NN regressor, training error **increases** as k (the number of neighbors looked at) increases.

You should think about **why** you observe each of the above phenomena. In particular, the last point may seem confusing – one would think that because larger values of k correspond to more complicated models (because the regressor is looking at more information to make a prediction), larger values of k should have lower training errors. But the nature of k-NN regressors is quite different than, say, decision tree regressors or linear regression models.

Lastly, in both cases, identify the ideal **hyperparameter** choice based on the graphs of testing error. You don't have to write the answer anywhere.

## Part 3: Predicting Survival on the Titanic 🛳🧊

### Question 6

Predicting whether or not passengers on the Titanic survived is a common first assignment when learning about classification – now it's your turn!

Complete the implementation of the function `titanic_model`, which takes in a DataFrame `titanic` containing **training data only** and returns a `Pipeline` object fit to the training data. 


#### Requirements

You have freedom to build your own model. That is, **you can use any classification algorithm**, but your model should satisfy the following requirements:

- The model is built on the (binary) response column `'Survived'`.
* The model uses features derived from **all other columns in `titanic`**. Below, we specify which columns to "engineer"; you may "engineer" features using other columns, but be sure to include every column in your model (even if you choose to leave some columns as-is).

* Required feature engineering:
    * Derive a feature from the "title" in the `'Name'` field (e.g. "Mr", "Miss", "Mrs" – the names themselves should not be used as a feature; think about why).
    * Derive a feature that standardizes passengers' ages among their `'Pclass'` (use Question 3!).
    
#### Evaluation
    
Your model must achieve an accuracy of 0.78 on both the training set and the test set. Note that while you have access to the test set, it is still encouraged to perform your own model validation.

**Extra credit: If your model can consistently earn an accuracy of above 0.83 on the test set, you can earn 5 points of extra credit on the lab!** That is, you can earn up to 60 points on the lab, even though it is only graded out of 55 points (meaning you _can_ have a score above 100% on the lab).

Some guidance:

- `Pipeline` objects can have other `Pipeline` objects within them. While this isn't a requirement, you may find this useful to break down your model into smaller, more manageable steps.

- You can make make `Pipeline` of just transformers.

- Your submitted `titanic_model` function should have the model's hyperparameters (e.g. tree depth) hard-coded in it. That is, the `Pipeline` object doesn't have to include the hyperparameter selection process.

- You will find [FunctionTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) useful. If you want your transformer to output a categorical feature, you will need to select `validate=False`.

- When using [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html#sklearn.compose.ColumnTransformer), you may find the `remainder` keyword helpful.

- If you are set out to get those extra 5 points, consider building some meaningful features before fine-tuning the hyperparameters of your model. Do an EDA on the dataset – what kinds of people are more prone to survive?

In [65]:
# Experiment using `titanic` below – remember, this is only your training data
titanic = pd.read_csv('data/titanic.csv')
titanic.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,2,Mr. William John Berriman,male,23.0,0,0,13.0000
1,1,3,Mrs. (Beila) Moor,female,27.0,0,1,12.4750
2,0,3,Mr. Nestor Cyriel Vande Walle,male,28.0,0,0,9.5000
3,0,3,Mr. Khalil Saad,male,25.0,0,0,7.2250
4,0,3,Miss. Gerda Ulrika Dahlberg,female,22.0,0,0,10.5167


In [66]:
from sklearn.model_selection import train_test_split
X = titanic.drop(columns = 'Survived')
y = titanic['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35)
pre = titanic_model(titanic)
(pre.predict(X_test)==y_test).mean()
# results = []
# for i in range(1000):
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35)
#     pre = titanic_model(titanic)
#     results.append((pre.predict(X_test)==y_test).mean())

0.8755020080321285

In [37]:
min(results)

(2.315733947782385, 0.5729929650348397)

In [38]:
(pre.predict(X_train)==y_train).mean()

0.8459869848156182

In [39]:
# don't change this cell, but do run it -- it is needed for the tests
q6_data_test = pd.read_csv('data/titanic.csv')
pl_test = titanic_model(q6_data_test)

In [40]:
q6_data_test

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,Title
0,0,2,Mr. William John Berriman,male,23.0,0,0,13.0000,Mr
1,1,3,Mrs. (Beila) Moor,female,27.0,0,1,12.4750,Mrs
2,0,3,Mr. Nestor Cyriel Vande Walle,male,28.0,0,0,9.5000,Mr
3,0,3,Mr. Khalil Saad,male,25.0,0,0,7.2250,Mr
4,0,3,Miss. Gerda Ulrika Dahlberg,female,22.0,0,0,10.5167,Miss
...,...,...,...,...,...,...,...,...,...
705,0,3,Mr. Raihed Razi,male,30.0,0,0,7.2292,Mr
706,0,2,Mr. John Henry Chapman,male,37.0,1,0,26.0000,Mr
707,1,2,Mrs. Sidney (Emily Hocking) Richards,female,24.0,2,3,18.7500,Mrs
708,0,3,Mr. Karl Gideon Gustafsson,male,19.0,0,0,7.7750,Mr


In [41]:
grader.check("q6")

q6 results: All test cases passed!

There is **a ton** of material out there on analyzing data from the Titanic. After you build your model, look online for other examples (e.g. [on Kaggle](https://www.kaggle.com/c/titanic)) and think about how you could improve your model.

## Congratulations! You've finished _the final lab of the quarter_! 🎉🥳

As a reminder, all of the work you want to submit needs to be in `lab.py`.

To verify that all of your work is indeed in `lab.py`, and that you didn't accidentally implement a function in this notebook and not in `lab.py`, we've included another notebook in the lab folder, called `lab-validation.ipynb`. `lab-validation.ipynb` is a version of this notebook with only the `grader.check` cells and the code needed to set up the tests. 

### **Go to `lab-validation.ipynb`, and go to Kernel > Restart & Run All.** This will check if all `grader.check` test cases pass using just the code in `lab.py`.

Once you're able to pass all test cases in `lab-validation.ipynb`, including the call to `grader.check_all()` at the very bottom, then you're ready to submit your `lab.py` (and only your `lab.py`) to Gradescope. Once submitting to Gradescope, make sure to stick around until all test cases pass.

There is also a call to `grader.check_all()` below in _this_ notebook, but make sure to also follow the steps above.

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [42]:
grader.check_all()

q1 results: All test cases passed!

q2 results: All test cases passed!

q3 results: All test cases passed!

q4 results: All test cases passed!

q5 results: All test cases passed!

q6 results: All test cases passed!